In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -q '/content/drive/MyDrive/Диплом/RetinaData.zip' -d '/content'

Mounted at /content/drive


In [ ]:
%pip install autokeras

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
TRAIN_PATH = '/content/Train'       # Папка для обучающего набора данных
TEST_PATH = '/content/Test'  # Папка для тестового набора данных
IMAGE_PATH = '/content/Train/'
TEST_SPLIT =0.1

In [ ]:
# Определение списка имен классов
CLASS_LIST = sorted(os.listdir(TRAIN_PATH))
# Определение количества классов
CLASS_COUNT = len(CLASS_LIST)
# Проверка результата
print(f'Количество классов: {CLASS_COUNT}, метки классов: {CLASS_LIST}')

Количество классов: 5, метки классов: ['0', '1', '2', '3', '4']


In [ ]:
data_files = []                           # Cписок путей к файлам картинок
data_labels = []                          # Список меток классов, соответствующих файлам

for class_label in range(CLASS_COUNT):    # Для всех классов по порядку номеров (их меток)
    class_name = CLASS_LIST[class_label]  # Выборка имени класса из списка имен
    class_path = IMAGE_PATH + class_name  # Формирование полного пути к папке с изображениями класса
    class_files = os.listdir(class_path)  # Получение списка имен файлов с изображениями текущего класса
    print(f'Размер класса {class_name} составляет {len(class_files)} фотографий')

    # Добавление к общему списку всех файлов класса с добавлением родительского пути
    data_files += [f'{class_path}/{file_name}' for file_name in class_files]

    # Добавление к общему списку меток текущего класса - их ровно столько, сколько файлов в классе
    data_labels += [class_label] * len(class_files)

print('Общий размер базы для обучения:', len(data_labels))

Размер класса 0 составляет 6408 фотографий
Размер класса 1 составляет 2532 фотографий
Размер класса 2 составляет 3502 фотографий
Размер класса 3 составляет 2879 фотографий
Размер класса 4 составляет 2709 фотографий
Общий размер базы для обучения: 18030


In [ ]:
!rm -rf Test

In [ ]:
# Перенос файлов для теста в отдельное дерево папок, расчет размеров наборов данных

os.mkdir(TEST_PATH)                                        # Создание папки для тестовых данных
train_count = 0
test_count = 0

for class_name in CLASS_LIST:                              # Для всех классов по порядку номеров (их меток)
    class_path = f'{TRAIN_PATH}/{class_name}'              # Формирование полного пути к папке с изображениями класса
    test_path = f'{TEST_PATH}/{class_name}'                # Полный путь для тестовых данных класса
    class_files = os.listdir(class_path)                   # Получение списка имен файлов с изображениями текущего класса
    class_file_count = len(class_files)                    # Получение общего числа файлов класса
    os.mkdir(test_path)                                    # Создание подпапки класса для тестовых данных
    test_file_count = int(class_file_count * TEST_SPLIT)   # Определение числа тестовых файлов для класса
    test_files = class_files[-test_file_count:]            # Выделение файлов для теста от конца списка
    for f in test_files:                                   # Перемещение тестовых файлов в папку для теста
        os.rename(f'{class_path}/{f}', f'{test_path}/{f}')
    train_count += class_file_count                        # Увеличение общего счетчика файлов обучающего набора
    test_count += test_file_count                          # Увеличение общего счетчика файлов тестового набора

    print(f'Размер класса {class_name}: {class_file_count} машин, для теста выделено файлов: {test_file_count}')

print(f'Общий размер базы: {train_count}, выделено для обучения: {train_count - test_count}, для теста: {test_count}')

Размер класса 0: 6408 машин, для теста выделено файлов: 640
Размер класса 1: 2532 машин, для теста выделено файлов: 253
Размер класса 2: 3502 машин, для теста выделено файлов: 350
Размер класса 3: 2879 машин, для теста выделено файлов: 287
Размер класса 4: 2709 машин, для теста выделено файлов: 270
Общий размер базы: 18030, выделено для обучения: 16230, для теста: 1800


In [ ]:
data_dir = '/content/Train'
batch_size = 24
image_size = (96,96)

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,                  # Путь к изображениям
  validation_split = 0.001,    # Устанавливаем разделение на обучающую и проверочную выборки (10%)
  subset = "training",       # Помечаем, что выборка обучающая (90%)
  shuffle = True,
  #color_mode ='grayscale',
  label_mode = 'int',
  seed = 123,                # Дополнительное случайное начальное число для перетасовки и преобразований
  image_size = image_size,   # Размер изображений, который был задан ранее
  batch_size = batch_size)   # Размер батча, который был задан ранее

#val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#  data_dir,                  # Путь к изображениям
#  validation_split = 0.001,    # Устанавливаем разделение на обучающую и проверочную выборки (10%)
#  color_mode='grayscale',
#  subset = "validation",     # Помечаем, что выборка проверочная (10%)
#  seed = 123,                # Дополнительное случайное начальное число для перетасовки и преобразований
#  shuffle = True,
#  image_size = image_size,   # Размер изображений, который был задан ранее
#  batch_size = batch_size)   # Размер батча, который был задан ранее

                             # Определяем имена классов:
class_names = train_ds.class_names
print(class_names)

Found 16230 files belonging to 5 classes.
Using 16214 files for training.
['0', '1', '2', '3', '4']


In [ ]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
train_ds.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
#val_ds = val_ds.map(process)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE) # Предварительная обучающая выборка
#val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)     # Предварительная тестовая выборка

print(train_ds)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [ ]:
import autokeras as ak
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduceLROnPlateau = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=3, verbose=1)

In [ ]:
clf = ak.ImageClassifier(num_classes=5,metrics='accuracy',overwrite=True, max_trials=3)

In [ ]:
#input_node = ak.ImageInput()
#output_node = ak.ImageBlock(
#    # Normalize the dataset.
#    normalize=True,
#    # Do not do data augmentation.
#    augment=False,
#)(input_node)
#output_node = ak.ClassificationHead()(output_node)
#clf = ak.AutoModel(
#    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
#)
clf.fit(
    train_ds,
    # Use your own validation set.
    validation_split=0.15,
    epochs=10,
    callbacks =[reduceLROnPlateau]
)

Trial 2 Complete [00h 43m 27s]
val_loss: 1.3015042543411255

Best val_loss So Far: 1.209535002708435
Total elapsed time: 00h 45m 01s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
efficient         |vanilla           |image_block_1/block_type
True              |True              |image_block_1/normalize
True              |False             |image_block_1/augment
True              |None              |image_block_1/image_augmentation_1/horizontal_flip
False             |None              |image_block_1/image_augmentation_1/vertical_flip
0                 |None              |image_block_1/image_augmentation_1/contrast_factor
0                 |None              |image_block_1/image_augmentation_1/rotation_factor
0.1               |None              |image_block_1/image_augmentation_1/translation_factor
0                 |None              |image_block_1/image_augmentation_1/zoom_factor
True              |None              |image_block_1/efficient_net_block

KeyboardInterrupt: 

In [ ]:
model = clf.export_model()
model.save('/content/drive/MyDrive/Диплом/Models/Autokeras.19.04.keras')